# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [11]:
# Find the establishments with a hygiene score of 20
query ={"scores.Hygiene" : {"$eq": 20}}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
results = establishments.find_one(query)
pprint(results)

41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65ff0ddac674b7f8996d0111'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractD

In [ ]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
hygscores_df = pd.json_normalize(results)

# hygscores_df.head()
# Display the number of rows in the DataFrame

print(len(hygscores_df))

# Display the first 10 rows of the DataFrame
hygscores_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue 
# greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
results = establishments.find_one(query)
pprint(results)

In [ ]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
londonscores_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(len(londonscores_df))

# Display the first 10 rows of the DataFrame
londonscores_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [33]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# PF's geocode is : lon 0.08384 , lat 51.490142
# so, search between lon 0.07384 and 0.09384, lat 51.480142 and 51.50142

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

pipeline = [
    {
        "$match": {
            "geocode.latitude": {
                "$gte": latitude - 0.01,
                "$lte": latitude + 0.01
            },
            "geocode.longitude": {
                "$gte": longitude - 0.01,
                "$lte": longitude + 0.01
            }
        }
    },
    {
        "$limit": 5
    },
    {
        "$sort": {
            "scores.Hygiene": 1
        }
    }
]
# Print the results
# results = establishments.find_one(query)
results = list(establishments.aggregate(pipeline))
pprint(results)

[{'AddressLine1': '152 Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'TIWA N TIWA African Restaurant Ltd',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'ChangesByServerID': 0,
  'Distance': 4646.930146257832,
  'FHRSID': 1069652,
  'LocalAuthorityBusinessID': 'PI/000206841',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 1JQ',
  'RatingDate': '2020-10-24T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('65ff0de7c674b7f8996d591c'),
  'geocode': {'latitude': 51.4870351, 'longitude': 0.0927429},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1069652',
             'rel': 'self'}],


In [35]:
# Convert result to Pandas DataFrame

nogonearpenang_df = pd.json_normalize(results)
nogonearpenang_df.head()

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,65ff0de7c674b7f8996d591c,1069652,0,PI/000206841,TIWA N TIWA African Restaurant Ltd,Restaurant/Cafe/Canteen,1,152 Plumstead High Street,,Plumstead,...,0.092743,51.487035,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,65ff0de7c674b7f8996d5943,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,0.092420,51.487148,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,65ff0de7c674b7f8996d5953,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,0.092537,51.487534,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,65ff0de7c674b7f8996d5965,695079,0,PI/000175160,Angels Bakery,Manufacturers/packers,7839,136 Plumstead High Street,,Plumstead,...,0.092267,51.487340,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,65ff0de7c674b7f8996d5964,1489059,0,15077,Kingsmeat Smokehouse,Other catering premises,7841,49 Lakedale Road,,Plumstead,...,0.091301,51.486377,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [45]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest


pipeline = [
    {'$match':{"scores.Hygiene": 0,} },
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Print the number of documents in the result
results = list(db.establishments.aggregate(pipeline))
resultscount = len(results)
print(resultscount)

# Print the first 10 results
pprint(results[0:10])


55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [44]:
# Convert the result to a Pandas DataFrame
markzero_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(len(markzero_df))
# Display the first 10 rows of the DataFrame
markzero_df.head(10)

55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
